In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from nucleosome_positioning import NPBackend
import triplib
from Bio import SeqIO, Seq
from Bio.Alphabet import IUPAC

# 2018-02-12 Tompitak model
Here I want to explore the possibility of using the bioinformatic model proposed by Tompiak et al. (BMC Bioinformatics (2017) 18:157 DOI 10.1186/s12859-017-1569-0) to evaluate the nucleosome positioning of sequences inserted in the Drosophila genome. The idea could be that maybe there is an effect due to this positioning signal in the expression of the TRIP reporters.

## Preliminaries: functions to perform the analyses

In [ ]:
nucpos_root_dir = '%s/work/CRG/projects/nucpos'%(os.getenv("HOME"))

In [ ]:
# define a function that inits the NP object which performs the calculations of the 
# probabilities, based on the "order" of the model and the "model".
def init_NP(order, model) :
    # constants
    datadir = '%s/code/MarkovModel/Probabilities/Nucleosomes/Room_Temperature'%nucpos_root_dir
    extensions = {1:'nucdist',2:'dinucdist',3:'trinucdist'}
    # Load in the probability tensor for the 'long' oligonucleotides
    filelong = '%s/%s.%s'%(datadir,model,extensions[order])
    rshptuplong = (148-order,) + (4,)*order
    Pl = np.genfromtxt(filelong).reshape(rshptuplong)
    # Load the probability tensor for the 'short' oligonucleotides if necessary,
    # otherwise init with order 1
    if (order > 1):
        fileshrt = '%s/%s.%s'%(datadir,model,extensions[order-1])
        rshptupshrt = (149-order,) + (4,)*(order-1)
        Ps = np.genfromtxt(fileshrt).reshape(rshptupshrt)
        # Set up the backend
        return NPBackend(order, 147, Pl, Ps)
    else:
        return NPBackend(order, 147, Pl)

In [ ]:
# init with a given model
orders = [2,3]
models = ['MD_RoomTemp','Mixed_RoomTemp','Olson_RoomTemp']
NP = {}
for order in orders :
    for model in models :
        NP[(order,model)] = init_NP(order, model)

In [ ]:
def load_sequence(seqfile,barcode) :
    """
    Loads a sequence files and subsitutes the 20 barcode nucleotides with the
    given 'barcode' sequence
    """
    with open (seqfile, "r") as myfile:
        seq = myfile.read()
    return seq.replace(20*'N',barcode)

## p8 promoter

I'll start the analysis with the p8 promoter.

In [ ]:
# load the sequence to examine
barcode_invented = 'ATGCTTTTTTGTACCCTGAA'
sequences_dir = '%s/data/sequences'%(nucpos_root_dir)
p8seq = load_sequence('%s/p8.seq'%(sequences_dir),barcode_invented)
print p8seq

We now proceed with a preliminary study: how does the different mechanical model of DNA and the "order" of the model affect the results, in terms of energy and probabilities of the configurations?

In [ ]:
p = {}
for order in orders :
    for model in models :
        p[(order,model)] = NP[(order,model)].ProbLandscape(p8seq)

In [ ]:
nmodels = len(models)
norders = len(orders)
N = nmodels * norders

In [ ]:
fig,axarr = plt.subplots(nmodels,norders,figsize=(15,9))
for i,model in enumerate(models) :
    for j,order in enumerate(orders) :
        ax = axarr[i,j]
        ax.semilogy(p[(order,model)])
        ax.set_title('Order = %d Model = %s'%(order,model))

So here I realize that the only things to compare are the "Olson" model and the "Mixed" models. The order doesn't matter very much, and the "MD" model is almost identical to the "Mixed" model.

Next, I want to look at how the neighboring sequence will affect the results of this analysis. I'll load the Drosophila Melanogaster genome, and take my sequence and insert it at various points, to see the difference.

In [ ]:
# load the Drosophila genome (using Biopython)
dm_genome_file = '/mnt/shared/seq/dm3R5/dmel-all-chromosome-r5.53_oneline.fasta'
dm = SeqIO.index(dm_genome_file, 'fasta', alphabet=IUPAC.unambiguous_dna)

In [ ]:
# get the sequence of the 2L chromosome, to start our investigation
chr2L = dm['2L'].seq

In [ ]:
# build the full sequence to analyze: left of "cut_site", inserted gene, 
# right of "cut_site"
def sequence_with_insertion(genome,left,right,cut_site,insertion) :
    return genome[cut_site-left:cut_site] +\
           insertion +\
           genome[cut_site:cut_site+right]

In [ ]:
left = 2000
right = 2000
cut_sites = [10235,984540]
sequences = {cut_site : sequence_with_insertion(chr2L,left,right,cut_site,p8seq)\
             for cut_site in cut_sites}

In [ ]:
# now run the modelling of the probability that the sequence binds nucleosomes
order = 2
mymodels = ['Mixed_RoomTemp','Olson_RoomTemp']
fullp = {}
for cut_site in cut_sites :
    for model in mymodels :
        fullp[(cut_site,model)] = NP[(order,model)].ProbLandscape(sequences[cut_site])

In [ ]:
xgene = np.arange(left,left+len(p8seq)-147+order-1)
fig,axarr = plt.subplots(2,2,figsize=(15,5))
for i,cut_site in enumerate(cut_sites) :
    for j,model in enumerate(mymodels) :
        ax = axarr[i,j]
        ax.semilogy(fullp[(cut_site,model)])
        ax.semilogy(xgene,p[(order,model)],'r')

In [ ]:
def energy(probability,kT=1.0) :
    return -kT * np.log(probability)

In [ ]:
Egene = {}
E = {}
order = 2
for model in mymodels :
    Egene[model] = energy(p[(order,model)])
    for cut_site in cut_sites :
        E[(cut_site,model)] = energy(fullp[(cut_site,model)])

In [ ]:
fig,axarr = plt.subplots(2,2,figsize=(15,5))
for i,cut_site in enumerate(cut_sites) :
    for j,model in enumerate(mymodels) :
        ax = axarr[i,j]
        ax.plot(E[(cut_site,model)])
        ax.plot(xgene,Egene[model],'r')

From this initial analysis there are a few things that emerge clearly. The clearest thing is that the neighboring sequences of the inserted gene do not have any effect on the probability/energy of the inserted gene. However, a crucial question remains: does the occupancy landscape change significantly? For this, I'll use Cédric's code to extract the occupancy profile from the energy profile. I'll save the energy profiles to files.

In [ ]:
# save files in a format that can be understood by Cédric's code
data_dir = '%s/data'%(nucpos_root_dir)
energy_profiles_dir = '%s/energy_profiles'%(data_dir)
chromosome = 'chr2L'
for key,profile in fullp.iteritems() :
    cutsite,model = key
    np.savetxt('%s/%s-%d-%d-%d-%s.dat'%(energy_profiles_dir,chromosome,cutsite,left,right,model),
               E[(cut_site,model)]-1000)

Now what remains to do is to feed the energy landscape to Cédric's code. However, this requires some additional workload because that code gives weird results, and is full of parameters, and I should find a way to invoke it from within here, otherwise it would become a real nightmare to deal with. Look at the pieces of code below, that I keep for the record.

I'll leave this on hold for a moment, and work on another problem before: go for an abstraction layer for my data handling, because otherwise it's going to be really difficult to keep track of all the parameters and stuff.

In [ ]:
def load_density_profile(chromosome,cut_site,left,right,model,landscape,excluded_volume,cutoff) :
    density_profiles_dir = '%s/density_profiles'%(data_dir)
    density_profile_file = '%s/%s-%d-%d-%d-%s-%s-%d-%d.dat'%(density_profiles_dir,
                                                             chromosome,
                                                             cut_site,
                                                             left,
                                                             right,
                                                             model,
                                                             landscape,
                                                             excluded_volume,
                                                             cutoff)
    return np.loadtxt(density_profile_file)

In [ ]:
chromosome = 'chr2L'
landscape = 'flat'
model = 'Mixed_RoomTemp'
cut_site = 984540
# cut_site = 10235
left = 2000
right = 2000
cutoff = 1
excluded_volume = 133
density_profiles = {}
density_profile = load_density_profile(chromosome,cut_site,left,right,model,
                                       landscape,excluded_volume,cutoff)

In [ ]:
figure = plt.figure(figsize=(15,5))
ax = plt.subplot(211)
ax.plot(density_profile)
ax.set_title('%s %d'%(landscape,cut_site))
ax = plt.subplot(212)
ax.plot(E[(cut_site,model)])